In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np
from scipy.misc import imread
from scipy.misc import imresize
from scipy.misc import imsave
import glob
import random

from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
#from keras.applications.resnet50 import preprocess_input

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.vgg19 import preprocess_input as preprocess_vgg
from keras.applications.inception_v3 import preprocess_input as inception_preprocess

#from keras.applications.inception_resnet_v2 import preprocess_input

import numpy as np

import keras
import copy
import cv2

from keras.models import *
from keras.layers import *
from keras.optimizers import *

import numpy as np 
import os
import numpy as np
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

Using TensorFlow backend.


In [3]:
path = '../data/'
savepath = '../data/'
images = glob.glob(path+'new_data/*.npy')
tile_sizes = np.loadtxt(path+'tile_sizes.txt', dtype='int')
images_sampled = {}
def loadData(window):
    global tile_sizes
    tile_sizes = tile_sizes[window-50:window]
    for tile in tile_sizes:
        for i in range(1, 31):
            images_sampled.setdefault(tile[0]*30+i, []).append(tile[1])

In [4]:
def myGenerator(batch_size):
    while True:
        index_list = random.sample(images_sampled.keys(), batch_size)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            frame = path+'sources/nparrs_384/frame'+str(i)+'.npy'
            frame = np.load(frame)
            tile_index = random.choice(images_sampled[i])
            
            temp  = imresize(frame[tile_index], (48, 48))
            temp  = imresize(temp, (384, 384))
            
            alldata_x.append(temp)
            alldata_y.append(frame[tile_index])
        
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        
        alldata_y = (alldata_y.astype(np.float32) - 127.5) / 127.5
        alldata_x = alldata_x.astype(np.float32)/255.0
        
        yield alldata_x, alldata_y

#x = myGenerator(10)
#xtrain, ytrain = next(x)
#print('xtrain shape:',xtrain.shape)
#print('ytrain shape:',ytrain.shape)

In [8]:
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 384
        self.img_cols = 384
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        #self.discriminator = self.build_discriminator()
        #self.discriminator.compile(loss=['mse'],
        #    optimizer=optimizer,
        #    metrics=['accuracy'])
        
        #print(self.discriminator.summary())
        
        # Build the generator
        self.generator = self.build_generator()
        
        print(self.generator.summary())
        
        noise = Input(shape=(384, 384, 3))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        #self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        #valid = self.discriminator(img)
        
        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        #self.combined = Model(noise, [img, valid])
        #self.combined.compile(loss=['mse', 'mse'],
        #    loss_weights=[0.9, 0.1],
        #    optimizer=optimizer)

    def build_generator(self):
        input_size = (384,384,3)
        inputs = Input(input_size)

        conv1 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv1 = LeakyReLU()(conv1)
        conv1 = BatchNormalization(momentum=0.8)(conv1)

        conv2 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(conv1)
        conv2 = LeakyReLU()(conv2)
        conv2 = BatchNormalization(momentum=0.8)(conv2)

        conv3 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(conv2)
        conv3 = LeakyReLU()(conv3)
        conv3 = BatchNormalization(momentum=0.8)(conv3)

        concat1 = add([conv1, conv3])

        conv4 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(concat1)
        conv4 = LeakyReLU()(conv4)
        conv4 = BatchNormalization(momentum=0.8)(conv4)

        conv5 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(conv4)
        conv5 = LeakyReLU()(conv5)
        conv5 = BatchNormalization(momentum=0.8)(conv5)

        concat2 = add([conv5, concat1])

        conv6 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(concat2)
        conv6 = LeakyReLU()(conv6)
        conv6 = BatchNormalization(momentum=0.8)(conv6)

        conv7 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(conv6)
        conv7 = LeakyReLU()(conv7)
        conv7 = BatchNormalization(momentum=0.8)(conv7)

        concat3 = add([conv7, concat2])

        #conv8 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(concat3)
        #conv8 = LeakyReLU()(conv8)
        #conv8 = BatchNormalization(momentum=0.8)(conv8)

        #conv9 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(conv8)
        #conv9 = LeakyReLU()(conv9)
        #conv9 = BatchNormalization(momentum=0.8)(conv9)
        

        #conv10 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(conv9)
        #conv10 = LeakyReLU()(conv10)
        #conv10 = BatchNormalization(momentum=0.8)(conv10)

        #concat4 = add([conv10, conv1])
        
        conv11 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(concat3)
        conv11 = LeakyReLU()(conv11)
        conv11 = BatchNormalization(momentum=0.8)(conv11)
        
        conv12 = Conv2D(8, 3, padding = 'same', kernel_initializer = 'he_normal')(conv11)
        conv12 = LeakyReLU()(conv12)
        conv12 = BatchNormalization(momentum=0.8)(conv12)


        out = Conv2D(3, 3, padding = 'same', kernel_initializer = 'he_normal')(conv12)
        out = LeakyReLU()(out)
        out = BatchNormalization(momentum=0.8)(out)


        model = Model(input = inputs, output = out)
        return model
    
    def build_autoencoder(self):
        self.generator.compile(loss=['mse'],optimizer=self.optimizer)
    
    def train_generator_autoencoder(self, epochs, dirname, batch_size=128, sample_interval=10):
        for epoch in range(epochs):
            X_train, y_train = next(myGenerator(batch_size))
            g_loss = self.generator.train_on_batch(X_train, y_train)
            if epoch % sample_interval == 0:
#                 #self.sample_images(epoch)
                 print ("Epoch ", epoch, " G loss ", g_loss)
#                 self.generator.save_weights(dirname+'/generator_weights_'+str(epoch)+'.h5')

    def sample_images(self, epoch):
        r, c             = 1, 10
        X_train, y_train = next(myGenerator(10))
        gen_imgs         = self.generator.predict(X_train)
        
        # Rescale images 0 - 1
        temp     = (0.5 * gen_imgs + 0.5)*255
        gen_imgs = temp.astype(int)
        y_train = (0.5 * y_train + 0.5)*255
        y_train = y_train.astype(int)
        X_train = X_train*255
        X_train = X_train.astype(int)
        
        combined = np.array([gen_imgs[0], gen_imgs[1], gen_imgs[2], gen_imgs[3], gen_imgs[4], gen_imgs[5], gen_imgs[6], gen_imgs[7], gen_imgs[8], gen_imgs[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+".png", combined)
        
        combined = np.array([y_train[0], y_train[1], y_train[2], y_train[3], y_train[4], y_train[5], y_train[6], y_train[7], y_train[8], y_train[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+"_real.png", combined)
        
        combined = np.array([X_train[0], X_train[1], X_train[2], X_train[3], X_train[4], X_train[5], X_train[6], X_train[7], X_train[8], X_train[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+"_lowres.png", combined)

In [ ]:
for i in range(27, 61):
    dirname = path+'Per-segment-models/'+str(i)
    loadData(i*50)
    #os.makedirs(dirname);
    cgan = CGAN()
    cgan.build_autoencoder()
    cgan.train_generator_autoencoder(20000, dirname, 2, 100)